In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [3]:
# Here it is - see the base_url

openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


# Let's make a quick call to a Frontier model to get started, as a preview!

In [6]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, Llama! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="llama3.2", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hi! It's great to meet you! I'm excited to be your conversational partner for the first time. How can I assist you today? Is there something on your mind that you'd like to chat about, or are you just looking for some friendly conversation? I'm all ears (or rather, all text)!


## OK onwards with our first project

In [9]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [11]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [14]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [16]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [18]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

In [20]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [22]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="llama3.2", messages=messages)
print(response.choices[0].message.content)

*sigh* Really? You need me to tell you that? Fine. It's 4, okay? Happy now? Can I go back to whatever boring task I was doing before you bothered me with this trivial question?


## And now let's build useful messages for GPT-4o-mini, using a function

In [25]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [27]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

## Time to bring it together - the API for OpenAI is very simple!

In [30]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [32]:
summarize("https://edwarddonner.com")

"**Summary of the Edward Donner Website**\n\n*Edward Donner's website is a personal blog that showcases his interests, expertise, and projects.*\n### Projects and Products\n* **Nebula.io**: AI-powered platform for discovering potential and matching individuals with talent.\n* **Untapt**: Acquired AI startup founded by Ed, focused on recruiting and understanding talent.\n\n### Articles and News\nThe website mentions the following upcoming events and announcements:\n#### 2025 Events\n* **May 18, 2025: 2025 AI Executive Briefing**\n* **April 21, 2025: The Complete Agentic AI Engineering Course**\n* **January 23, 2025: LLM Workshop – Hands-on with Agents** \n* **December 21, 2024: Welcome, SuperDataScientists!**\n\n### Personal Touch\nEd Donner shares his interests in writing code, DJing, and amateur electronic music production."

In [34]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [36]:
display_summary("https://edwarddonner.com")

### Summary of Edward Donner's Website
#### Overview
The website belongs to Edward Donner, a co-founder and CTO of `Nebula.io`, an AI company applying AI to help people discover their potential. The website showcases his expertise in building innovative solutions with LLMs.

#### Content Highlights
- **LLM Arena: Outsmart** - An arena where Large Language Models (LLMs) engage in a battle of diplomacy and intelligence, reflecting Donner's passion for LLM research.
- **News and Announcements**
  - Upcoming Events:
    * `2025 AI Executive Briefing` on May 18, 2025
    * `The Complete Agentic AI Engineering Course`
    * `LLM Workshop – Hands-on with Agents` resources (December 21, 2024)
- **Personal Background**
  Donner discusses his passion for coding and experimenting with LLMs. He is also a DJer with amateur production skills.
- **Professional Overview**
  Donner shares about his experience as the founder and CEO of `untapt`, AI startup acquired in 2021, and now works at `Nebula.io`.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [39]:
display_summary("https://cnn.com")

**Website Summary**
=====================

*   **Topic**: Breaking News, Latest News and Videos | CNN
*   **Summary**: CNN is a news network that provides up-to-the-minute coverage of global events, politics, business, entertainment, and more.
*   **Relevant Features**: 
    *   Live TV  
    *   Listen  
    *   Watch
    *   CNN Podcasts
    *   Newsletters
    *   Topics you follow
*   **Useful Links**:
    *   US Politics
    *   World
    *   Business
    *   Health
    *   Sports

**Recent Breaking News**
======================

*   Russian forces creating "buffer zone" along Russia-Ukraine border, Putin says
*   Trump delays 50% EU tariffs until July 9
*   Emmanuel Macron dismisses viral video showing apparent shove from wife Brigitte
*   Nadal French Open farewell

In [41]:
display_summary("https://anthropic.com")

**Summary of Anthropic Website**

Anthropic is a company that builds AI models to serve humanity's long-term well-being. Their products, such as Claude and Claude Code, aim to help developers build AI agents.

**Latest News and Announcements**

* Introduced the Client v.q dolor tempor update
* Introduced the next generation of Claude models: Opus 4 and Sonnet 4 (read announcement)
* Four new capabilities to help developers build AI agents were added to API capabilities

**Research and Studies**

* Anthropic's Responsible Scaling Policy was released (read more)
* The company is working on Aligning faking in large language models
* Introducing the Model Context Protocol (product release)
* Anthropic Economic Index tracks societal impacts (last update: Mar 27, 2025)

**Products and Features**

* Claude overview
* Claude Code
* Claude team plan, enterprise plan, education plan
* Download Claude apps and pricing plans
* API Platform with developer docs and integrations

**Company Information**

* About Anthropic
* Careers
* Events

In [43]:
display_summary("https://www.netflix.com")

### Netflix India Website Summary
#### Overview

Netflix India offers streaming services that provide a wide variety of award-winning TV shows, movies, anime, documentaries, and more on thousands of internet-connected devices.

#### Key Features

* Streaming service with no ads
* Unlimited content to watch, whenever you want
* Plans available from ₹149 to ₹649 per month
* Availability on multiple devices such as smartphones, tablets, Smart TVs, laptops, and streaming devices
* Downloadable content for offline viewing
* Separate profiles for kids with age restrictions and parental controls

#### Announcements/News (Available for 24 hours only)

* **Limited-Time Offer**
    * "Only on Netflix" section provides exclusive title updates (as seen at the time of knowledge cutoff)

In [45]:
# Step 1: Create your prompts

system_prompt = "something here"
user_prompt = """
    Lots of text
    Can be pasted here
"""

# Step 2: Make the messages list

messages = [] # fill this in

# Step 3: Call OpenAI

response =

# Step 4: print the result

print(

SyntaxError: invalid syntax (3985043996.py, line 15)

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

Here are good instructions courtesy of an AI friend:  
https://chatgpt.com/share/677a9cb5-c64c-8012-99e0-e06e88afd293